In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%ls

'Copy of real_question.json'   questions_Jason.csv
 document_parsing.ipynb        questions.json
 expo_ques_pair.ipynb          questions_labeled.json
 filtered_text.json            raw_data.json
 questions_Eric.csv            test.ipynb
 questions_Eric_labeled.csv    Topic_Modeling.ipynb


In [ ]:
%cd ResearchSALT/

/content/drive/MyDrive/Colab Notebooks/ResearchSALT


In [ ]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/ResearchSALT'

In [ ]:
import json
f = open("/content/drive/MyDrive/Colab Notebooks/ResearchSALT/raw_data.json",)
data = json.load(f)

print(len(data))
f.close()

5326


In [ ]:
print(data[17])

In [ ]:
#Further filtering

#1. segment the data
data_seg = []
for doc in data:
  data_seg.append(list(doc.split('\n')))

#2. filtering out data without proper participant list
filtered_data_seg = []
for doc in data_seg:
  if 'Operator' in doc:
    if ("Executives" and "Analysts" in doc) or ("Company Participants" and "Conference Call Participants" in doc) and ("Question-and-Answer Session" or "Q&A Session"):
      filtered_data_seg.append(doc)

print(len(filtered_data_seg))

5296


In [ ]:
#rejoin the segments
filtered_data = []

for doc in filtered_data_seg:
  true_doc = '\n'.join(doc)
  filtered_data.append(true_doc)
print(len(filtered_data))
#print(filtered_data[0])

5296


In [ ]:
#executives list
#list of the title
exec_list = []
titl_list = []
for conference in filtered_data_seg:
  if "Executives" in conference:
    exec_pos = conference.index("Executives")
    anlst_pos = conference.index("Analysts")
  elif "Company Participants" in conference:
    exec_pos = conference.index("Company Participants")
    anlst_pos = conference.index("Conference Call Participants")
  exec_list.append(conference[exec_pos:anlst_pos])
  titl_list.append(conference[0:exec_pos])
print(len(exec_list))

5296


In [ ]:
#analysts list
anlst_list = []
for conference in filtered_data_seg:
  if "Analysts" in conference:
    start_pos = conference.index("Analysts")
    end_pos = conference.index("Operator")
  elif "Company Participants" in conference:
    start_pos = conference.index("Conference Call Participants")
    end_pos = conference.index("Operator")
  anlst_list.append(conference[start_pos:end_pos])
print(len(anlst_list))

5296


In [ ]:
#q_&_a_list and expo list
data_qa = []
data_expo = []
for conference in filtered_data_seg:
  if 'Question-and-Answer Session' in conference:
    qa_pos = conference.index('Question-and-Answer Session')
  elif 'Q&A Session' in conference:
    qa_pos = conference.index('Q&A Session')
  qa = conference[qa_pos:]
  expo = conference[0:qa_pos]
  data_qa.append(qa)
  data_expo.append(expo)

print(len(data_qa))
print(len(data_expo))

5296
5296


In [ ]:
def is_exec(seg, executives):
  isExec = False
  for exec in executives:
    if seg in exec:
      isExec = True
  return isExec

In [ ]:
def is_anlst(seg, analysts):
  isAnlst = False
  for anlst in analysts:
    if seg in anlst:
      isAnlst = True
  return isAnlst

In [ ]:
#question parsing analysts' utterance
def parseAnaUtt(list_seg, executives, analysts):
  Utt_list = []
  Q_A_pair = []
  for i in range(len(list_seg)):

    if is_anlst(list_seg[i], analysts):
      start_pos = i
      end_pos = i

      #ana_nam = list_seg[i]
      j = i + 1
      for j in range(i + 1, len(list_seg)):
        if is_exec(list_seg[j], executives) or is_anlst(list_seg[j], analysts) or list_seg[j] == "Operator" or j == len(list_seg):
          end_pos = j
          break
      if j != len(list_seg) and is_exec(list_seg[j], executives):
        answer_end_pos = j
        for k in range(j, len(list_seg)):
          if is_anlst(list_seg[k], analysts) or list_seg[k] == "Operator" or k == len(list_seg):
            answer_end_pos = k
            break
        Q_A_pair.append([ '\n'.join(list_seg[(start_pos + 1):end_pos]) , '\n'.join(list_seg[(end_pos + 1):answer_end_pos]) ])
      Utt_list.extend(list_seg[(start_pos + 1):end_pos])
      i = j
  
  return Utt_list, Q_A_pair

In [ ]:
#question parsing executives' utterance
def parseExecUtt(list_seg, executives, analysts):
  Utt_list = []
  for i in range(len(list_seg)):

    if is_exec(list_seg[i], executives):
      start_pos = i
      end_pos = i
    
      #exe_nam = list_seg[i]
      j = i + 1
      for j in range(i + 1, len(list_seg)):
        if is_exec(list_seg[j], executives) or is_anlst(list_seg[j], analysts) or list_seg[j] == "Operator" or j == len(list_seg):
          end_pos = j
          break
      Utt_list.extend(list_seg[start_pos + 1:end_pos])
      i = j
  
  return Utt_list

In [ ]:
utterance_parse = []
count = 0
for i in range(len(filtered_text_seg)):
  try:
    q = parseQ_pure(filtered_text_seg[i], exec_list[i], anlst_list[i])
#    if ' ' in q:
#      q = q.remove(' ')
    emp_list = []
    for a in q:
      if a == emp_list:
        q.remove(a)
    e = parseExp_pure(filtered_text_seg[i], exec_list[i], anlst_list[i])
    for a in e:
      if a == emp_list:
        e.remove(a)
#    if ' ' in e:
#      e = e.remove(' ')
    utterance_parse.append((q,e))
#UnboundLocalError, 
  except UnboundLocalError :
      count += 1
      #print(count)
      pass

print(len(utterance_parse))

NameError: ignored

In [ ]:
from pprint import pprint

In [ ]:
anaUtts = []
Q_A_pairs = []
for i in range(len(data_qa)):
  anaUtt, Q_A = parseAnaUtt(data_qa[i], exec_list[i], anlst_list[i])
  anaUtts.append(anaUtt)
  Q_A_pairs.append(Q_A)
print(len(anaUtts))

5296


In [ ]:
pprint(Q_A_pairs[0])

[['Thanks very much operator. Hi everybody. Thank you very much for your time. '
  'Just on the bookings over the past -- I mean, this is like very '
  'short-term-focused, but are you hearing from any of your corporate accounts '
  "about adjusting bookings or whatever for the -- given United's performance "
  'over the last couple of weeks with various incidences?',
  "We had a lot of communication with our corporate accounts, and we've had "
  'appropriate questions and concerns, particularly with regard to our '
  "corporate accounts, with whom we have a good relationship and we've already "
  "communicated with. We feel pretty good about the communications that we've "
  'had so far and in our ability to reassure them and explain things like '
  'overbooking.\n'
  'Some of the steps that we will do with the broader public on our April 30 '
  "launch but some of that with our corporate accounts, we've been able to "
  'accelerate some of that -- some of their specific concerns. And

In [ ]:
exeUtts = []
for i in range(len(data_expo)):
  exeUtt = parseExecUtt(data_expo[i], exec_list[i], anlst_list[i])
  exeUtts.append(exeUtt)
print(len(exeUtts))
#pprint(exeUtts)

5296


In [ ]:
data_q_a_pairs = []
for i in range(len(data_expo)):
  dic = {}
  dic['title'] = titl_list[i]
  dic['executives'] = exec_list[i]
  dic['analysts'] = anlst_list[i]
  dic['expo'] = data_expo[i]
  dic['Q_A_pairs'] = Q_A_pairs[i]
  data_q_a_pairs.append(dic)


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ResearchSALT/data_qa_pairs.json', 'w') as f: 
      
    # using csv.writer method from CSV package 
    json.dump(data_q_a_pairs, f)
    f.close()

In [ ]:
pprint(data_q_a_pairs[0]['Q_A_pairs'])

[[['Thanks very much operator. Hi everybody. Thank you very much for your '
   'time. Just on the bookings over the past -- I mean, this is like very '
   'short-term-focused, but are you hearing from any of your corporate '
   "accounts about adjusting bookings or whatever for the -- given United's "
   'performance over the last couple of weeks with various incidences?'],
  ["We had a lot of communication with our corporate accounts, and we've had "
   'appropriate questions and concerns, particularly with regard to our '
   "corporate accounts, with whom we have a good relationship and we've "
   'already communicated with. We feel pretty good about the communications '
   "that we've had so far and in our ability to reassure them and explain "
   'things like overbooking.',
   'Some of the steps that we will do with the broader public on our April 30 '
   "launch but some of that with our corporate accounts, we've been able to "
   'accelerate some of that -- some of their specific

In [ ]:
output = [exeUtts, anaUtts]

In [ ]:
import csv
import json

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ResearchSALT/paragraph_pairs_5296.json', 'w') as f: 
      
    # using csv.writer method from CSV package 
    json.dump(output, f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ResearchSALT/analyst_utterance_5296.json', 'w') as f: 
      
    # using csv.writer method from CSV package 
    json.dump(anaUtts, f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ResearchSALT/exposition_utterance_5296.json', 'w') as f: 
      
    # using csv.writer method from CSV package 
    json.dump(exeUtts, f)

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/ResearchSALT/question_answer_pairs_5296.json', 'w') as f: 
      
    # using csv.writer method from CSV package 
    json.dump(Q_A_pairs, f)